In [1]:
%%writefile DaGAlgo.cpp
#include <iostream>
#include <vector>
#include <fstream>
#include <array>
#include <iomanip>  // For setprecision
#include <chrono>   // For timing
#include <algorithm>  // For min/max

using namespace std;

using Vec3 = std::array<double, 3>;
using Triangle = std::array<Vec3, 3>;
constexpr double EPSILON = 1e-12;  // Higher precision for tolerance check

// Helper function: Compute determinant
double determinant(const Vec3& a, const Vec3& b, const Vec3& c, const Vec3& d) {
    return (a[0] - d[0]) * ((b[1] - d[1]) * (c[2] - d[2]) - (c[1] - d[1]) * (b[2] - d[2])) -
           (a[1] - d[1]) * ((b[0] - d[0]) * (c[2] - d[2]) - (c[0] - d[0]) * (b[2] - d[2])) +
           (a[2] - d[2]) * ((b[0] - d[0]) * (c[1] - d[1]) - (c[0] - d[0]) * (b[1] - d[1]));
}

// Devillers & Guigue Algorithm for Triangle-Triangle Intersection
bool devillersGuigueIntersection(const Triangle& T1, const Triangle& T2) {
    double d1 = determinant(T2[0], T2[1], T2[2], T1[0]);
    double d2 = determinant(T2[0], T2[1], T2[2], T1[1]);
    double d3 = determinant(T2[0], T2[1], T2[2], T1[2]);

    // If all three determinants have the same sign, triangles do not intersect
    if ((d1 > 0 && d2 > 0 && d3 > 0) || (d1 < 0 && d2 < 0 && d3 < 0)) {
        return false;
    }

    double d4 = determinant(T1[0], T1[1], T1[2], T2[0]);
    double d5 = determinant(T1[0], T1[1], T1[2], T2[1]);
    double d6 = determinant(T1[0], T1[1], T1[2], T2[2]);

    // If all three determinants have the same sign, triangles do not intersect
    if ((d4 > 0 && d5 > 0 && d6 > 0) || (d4 < 0 && d5 < 0 && d6 < 0)) {
        return false;
    }

    return true; // Otherwise, triangles intersect
}

// Function to read triangles from an OFF file
std::vector<Triangle> readTrianglesFromOFF(const std::string& filename) {
    std::ifstream infile(filename);
    if (!infile) {
        std::cerr << "Error opening file: " << filename << std::endl;
        exit(1);
    }

    std::string header;
    infile >> header;
    if (header != "OFF") {
        std::cerr << "Not a valid OFF file." << std::endl;
        exit(1);
    }

    int numVertices, numFaces, numEdges;
    infile >> numVertices >> numFaces >> numEdges;

    std::vector<Vec3> vertices(numVertices);
    for (int i = 0; i < numVertices; ++i) {
        infile >> vertices[i][0] >> vertices[i][1] >> vertices[i][2];
    }

    std::vector<Triangle> triangles;
    for (int i = 0; i < numFaces; ++i) {
        int vertexCount, v1, v2, v3;
        infile >> vertexCount >> v1 >> v2 >> v3;
        if (vertexCount == 3) {
            triangles.push_back({vertices[v1], vertices[v2], vertices[v3]});
        }
    }

    return triangles;
}

int main() {
    auto start  = std::chrono::high_resolution_clock::now();

    auto triangles1 = readTrianglesFromOFF("VH_M_ileocecal_valve.off");
    auto triangles2 = readTrianglesFromOFF("VH_M_transverse_colon.off");

    bool intersect = false;
    for (const auto& tri1 : triangles1) {
        for (const auto& tri2 : triangles2) {
            if (devillersGuigueIntersection(tri1, tri2)) {
                intersect = true;
                // Print the intersecting triangles
                std::cout << "Triangle 1: " << tri1[0][0] << " " << tri1[0][1] << " " << tri1[0][2] << std::endl;
                std::cout << "Triangle 2: " << tri2[0][0] << " " << tri2[0][1] << " " << tri2[0][2] << std::endl;
                break;
            }
        }
        if (intersect) break;
    }

    if (intersect) {
        std::cout << "Triangles intersect!" << std::endl;
    } else {
        std::cout << "Triangles do not intersect." << std::endl;
    }

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end - start;
    cout << "Time taken: " << elapsed.count() << " seconds" << endl;

    return 0;
}


Writing DaGAlgo.cpp


In [2]:
!g++ DaGAlgo.cpp -o DaGAlgo

In [3]:
!./DaGAlgo

Triangle 1: -0.0448705 0.175102 0.0555511
Triangle 2: -0.0241759 0.245838 0.0580359
Triangles intersect!
Time taken: 0.0318319 seconds


In [ ]:
%%writefile DaGAlgoGPU.cu
#include <iostream>
#include <vector>
#include <fstream>
#include <array>
#include <chrono>
#include <cuda_runtime.h>

using namespace std;

using Vec3 = array<double, 3>;
using Triangle = array<Vec3, 3>;
constexpr double EPSILON = 1e-12;

// ======================================================================
// Devillers & Guigue Determinant Function
// ======================================================================

_device_ double determinant(const Vec3& a, const Vec3& b, const Vec3& c, const Vec3& d) {
    return (a[0] - d[0]) * ((b[1] - d[1]) * (c[2] - d[2]) - (c[1] - d[1]) * (b[2] - d[2])) -
           (a[1] - d[1]) * ((b[0] - d[0]) * (c[2] - d[2]) - (c[0] - d[0]) * (b[2] - d[2])) +
           (a[2] - d[2]) * ((b[0] - d[0]) * (c[1] - d[1]) - (c[0] - d[0]) * (b[1] - d[1]));
}

// ======================================================================
// Devillers & Guigue Triangle-Triangle Intersection
// ======================================================================

_device_ bool devillersGuigueIntersection(const Triangle& T1, const Triangle& T2) {
    double d1 = determinant(T2[0], T2[1], T2[2], T1[0]);
    double d2 = determinant(T2[0], T2[1], T2[2], T1[1]);
    double d3 = determinant(T2[0], T2[1], T2[2], T1[2]);

    // If all determinants have the same sign, T1 is fully on one side of T2
    if ((d1 > 0 && d2 > 0 && d3 > 0) || (d1 < 0 && d2 < 0 && d3 < 0)) {
        return false;
    }

    double d4 = determinant(T1[0], T1[1], T1[2], T2[0]);
    double d5 = determinant(T1[0], T1[1], T1[2], T2[1]);
    double d6 = determinant(T1[0], T1[1], T1[2], T2[2]);

    // If all determinants have the same sign, T2 is fully on one side of T1
    if ((d4 > 0 && d5 > 0 && d6 > 0) || (d4 < 0 && d5 < 0 && d6 < 0)) {
        return false;
    }

    return true; // Otherwise, the triangles intersect
}

// ======================================================================
// GPU Kernel: Triangle-Triangle Intersection Check
// ======================================================================

_global_ void triangleIntersectionKernel(const Triangle* triangles1, int numTriangles1,
                                           const Triangle* triangles2, int numTriangles2,
                                           bool* intersectionFound) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = idx; i < numTriangles1 && !(*intersectionFound); i += stride) {
        for (int j = 0; j < numTriangles2 && !(*intersectionFound); ++j) {
            if (devillersGuigueIntersection(triangles1[i], triangles2[j])) {
                *intersectionFound = true;
            }
        }
    }
}

// ======================================================================
// Function to Read Triangles from an OFF File
// ======================================================================

vector<Triangle> readTrianglesFromOFF(const string& filename) {
    ifstream infile(filename);
    if (!infile) {
        cerr << "Error opening file: " << filename << endl;
        exit(1);
    }

    string header;
    infile >> header;
    if (header != "OFF") {
        cerr << "Not a valid OFF file." << endl;
        exit(1);
    }

    int numVertices, numFaces, numEdges;
    infile >> numVertices >> numFaces >> numEdges;

    vector<Vec3> vertices(numVertices);
    for (int i = 0; i < numVertices; ++i) {
        infile >> vertices[i][0] >> vertices[i][1] >> vertices[i][2];
    }

    vector<Triangle> triangles;
    triangles.reserve(numFaces);
    for (int i = 0; i < numFaces; ++i) {
        int vertexCount, v1, v2, v3;
        infile >> vertexCount >> v1 >> v2 >> v3;
        if (vertexCount == 3) {
            triangles.push_back({vertices[v1], vertices[v2], vertices[v3]});
        } else {
            for (int skip = 0; skip < vertexCount - 3; skip++) {
                infile >> v1;
            }
        }
    }
    return triangles;
}

// ======================================================================
// Main Function
// ======================================================================

int main() {
    auto start = chrono::high_resolution_clock::now();

    auto triangles1 = readTrianglesFromOFF("VH_M_ileocecal_valve.off");
    auto triangles2 = readTrianglesFromOFF("VH_M_transverse_colon.off");

    Triangle *d_triangles1, *d_triangles2;
    bool *d_intersectionFound;

    cudaMalloc(&d_triangles1, triangles1.size() * sizeof(Triangle));
    cudaMalloc(&d_triangles2, triangles2.size() * sizeof(Triangle));
    cudaMalloc(&d_intersectionFound, sizeof(bool));

    cudaMemcpy(d_triangles1, triangles1.data(), triangles1.size() * sizeof(Triangle), cudaMemcpyHostToDevice);
    cudaMemcpy(d_triangles2, triangles2.data(), triangles2.size() * sizeof(Triangle), cudaMemcpyHostToDevice);

    bool intersectionFound = false;
    cudaMemcpy(d_intersectionFound, &intersectionFound, sizeof(bool), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (static_cast<int>(triangles1.size()) + blockSize - 1) / blockSize;

    triangleIntersectionKernel<<<numBlocks, blockSize>>>(d_triangles1,
                                                         static_cast<int>(triangles1.size()),
                                                         d_triangles2,
                                                         static_cast<int>(triangles2.size()),
                                                         d_intersectionFound);

    cudaMemcpy(&intersectionFound, d_intersectionFound, sizeof(bool), cudaMemcpyDeviceToHost);

    cudaFree(d_triangles1);
    cudaFree(d_triangles2);
    cudaFree(d_intersectionFound);

    if (intersectionFound) {
        cout << "Triangles intersect!" << endl;
    } else {
        cout << "Triangles do not intersect." << endl;
    }

    auto end = chrono::high_resolution_clock::now();
    chrono::duration<double> elapsed = end - start;
    cout << "Time taken: " << elapsed.count() << " seconds" << endl;

    return 0;
}

Overwriting DaGAlgoGPU.cu


In [7]:
!nvcc DaGAlgoGPU.cu -o DaGAlgoGPU --expt-relaxed-constexpr

In [8]:
!./DaGAlgoGPU

Triangles do not intersect.
Time taken: 0.21738 seconds


In [9]:
# Shane's Algo

In [10]:
%%writefile Shane.cpp
#include <iostream>
#include <vector>
#include <fstream>
#include <array>
#include <iomanip>
#include <chrono>
#include <algorithm>

using namespace std;

using Vec3 = std::array<double, 3>;
using Triangle = std::array<Vec3, 3>;
constexpr double EPSILON = 1e-12;

// Helper function: Cross product
Vec3 cross(const Vec3& a, const Vec3& b) {
    return {
        a[1] * b[2] - a[2] * b[1],
        a[2] * b[0] - a[0] * b[2],
        a[0] * b[1] - a[1] * b[0]
    };
}

// Helper function: Dot product
double dot(const Vec3& a, const Vec3& b) {
    return a[0] * b[0] + a[1] * b[1] + a[2] * b[2];
}

// Helper function: Vector subtraction
Vec3 subtract(const Vec3& a, const Vec3& b) {
    return {a[0] - b[0], a[1] - b[1], a[2] - b[2]};
}

// Compute the normal of a triangle
Vec3 computeNormal(const Triangle& tri) {
    return cross(subtract(tri[1], tri[0]), subtract(tri[2], tri[0]));
}

// Compute signed distance from a point to a plane
double signedDistance(const Vec3& p, const Vec3& normal, const Vec3& p0) {
    return dot(normal, subtract(p, p0));
}

// **Shen's Algorithm** for Triangle-Triangle Intersection
bool shenIntersection(const Triangle& T1, const Triangle& T2) {
    Vec3 normal1 = computeNormal(T1);
    Vec3 normal2 = computeNormal(T2);

    // Compute signed distances
    double d1 = signedDistance(T2[0], normal1, T1[0]);
    double d2 = signedDistance(T2[1], normal1, T1[0]);
    double d3 = signedDistance(T2[2], normal1, T1[0]);

    double d4 = signedDistance(T1[0], normal2, T2[0]);
    double d5 = signedDistance(T1[1], normal2, T2[0]);
    double d6 = signedDistance(T1[2], normal2, T2[0]);

    // If all points of one triangle are on the same side of the separation plane, return false
    if ((d1 > 0 && d2 > 0 && d3 > 0) || (d1 < 0 && d2 < 0 && d3 < 0)) {
        return false;
    }

    if ((d4 > 0 && d5 > 0 && d6 > 0) || (d4 < 0 && d5 < 0 && d6 < 0)) {
        return false;
    }

    return true;
}

// Function to read triangles from an OFF file
std::vector<Triangle> readTrianglesFromOFF(const std::string& filename) {
    std::ifstream infile(filename);
    if (!infile) {
        std::cerr << "Error opening file: " << filename << std::endl;
        exit(1);
    }

    std::string header;
    infile >> header;
    if (header != "OFF") {
        std::cerr << "Not a valid OFF file." << std::endl;
        exit(1);
    }

    int numVertices, numFaces, numEdges;
    infile >> numVertices >> numFaces >> numEdges;

    std::vector<Vec3> vertices(numVertices);
    for (int i = 0; i < numVertices; ++i) {
        infile >> vertices[i][0] >> vertices[i][1] >> vertices[i][2];
    }

    std::vector<Triangle> triangles;
    for (int i = 0; i < numFaces; ++i) {
        int vertexCount, v1, v2, v3;
        infile >> vertexCount >> v1 >> v2 >> v3;
        if (vertexCount == 3) {
            triangles.push_back({vertices[v1], vertices[v2], vertices[v3]});
        }
    }

    return triangles;
}

int main() {
    auto start  = std::chrono::high_resolution_clock::now();

    auto triangles1 = readTrianglesFromOFF("VH_M_ileocecal_valve.off");
    auto triangles2 = readTrianglesFromOFF("VH_M_transverse_colon.off");

    bool intersect = false;
    for (const auto& tri1 : triangles1) {
        for (const auto& tri2 : triangles2) {
            if (shenIntersection(tri1, tri2)) {
                intersect = true;
                // Print the intersecting triangles
                std::cout << "Triangle 1: " << tri1[0][0] << " " << tri1[0][1] << " " << tri1[0][2] << std::endl;
                std::cout << "Triangle 2: " << tri2[0][0] << " " << tri2[0][1] << " " << tri2[0][2] << std::endl;
                break;
            }
        }
        if (intersect) break;
    }

    if (intersect) {
        std::cout << "Triangles intersect!" << std::endl;
    } else {
        std::cout << "Triangles do not intersect." << std::endl;
    }

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end - start;
    cout << "Time taken: " << elapsed.count() << " seconds" << endl;

    return 0;
}


Writing Shane.cpp


In [11]:
!g++ Shane.cpp -o Shane

In [13]:
!./Shane

Triangle 1: -0.0448705 0.175102 0.0555511
Triangle 2: -0.0241759 0.245838 0.0580359
Triangles intersect!
Time taken: 0.0331914 seconds


In [14]:
%%writefile Shane.cu
#include <iostream>
#include <vector>
#include <fstream>
#include <array>
#include <chrono>
#include <cuda_runtime.h>

using namespace std;

using Vec3 = array<double, 3>;
using Triangle = array<Vec3, 3>;
constexpr double EPSILON = 1e-12;

// CUDA Helper Functions
__device__ Vec3 cross(const Vec3& a, const Vec3& b) {
    return {
        a[1] * b[2] - a[2] * b[1],
        a[2] * b[0] - a[0] * b[2],
        a[0] * b[1] - a[1] * b[0]
    };
}

__device__ double dot(const Vec3& a, const Vec3& b) {
    return a[0] * b[0] + a[1] * b[1] + a[2] * b[2];
}

__device__ Vec3 subtract(const Vec3& a, const Vec3& b) {
    return {a[0] - b[0], a[1] - b[1], a[2] - b[2]};
}

// Compute the normal of a triangle
__device__ Vec3 computeNormal(const Triangle& tri) {
    return cross(subtract(tri[1], tri[0]), subtract(tri[2], tri[0]));
}

// Compute signed distance from a point to a plane
__device__ double signedDistance(const Vec3& p, const Vec3& normal, const Vec3& p0) {
    return dot(normal, subtract(p, p0));
}

// **Shen's Algorithm** for CUDA Parallel Execution
__device__ bool shenIntersection(const Triangle& T1, const Triangle& T2) {
    Vec3 normal1 = computeNormal(T1);
    Vec3 normal2 = computeNormal(T2);

    // Compute signed distances for plane separation test
    double d1 = signedDistance(T2[0], normal1, T1[0]);
    double d2 = signedDistance(T2[1], normal1, T1[0]);
    double d3 = signedDistance(T2[2], normal1, T1[0]);

    double d4 = signedDistance(T1[0], normal2, T2[0]);
    double d5 = signedDistance(T1[1], normal2, T2[0]);
    double d6 = signedDistance(T1[2], normal2, T2[0]);

    // If all points of one triangle are on the same side of the plane, they do not intersect
    if ((d1 > 0 && d2 > 0 && d3 > 0) || (d1 < 0 && d2 < 0 && d3 < 0)) {
        return false;
    }
    if ((d4 > 0 && d5 > 0 && d6 > 0) || (d4 < 0 && d5 < 0 && d6 < 0)) {
        return false;
    }

    return true;
}

// CUDA Kernel for Triangle-Triangle Intersection
__global__ void triangleIntersectionKernel(const Triangle* triangles1, int numTriangles1,
                                           const Triangle* triangles2, int numTriangles2,
                                           bool* intersectionFound) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = idx; i < numTriangles1 && !(*intersectionFound); i += stride) {
        for (int j = 0; j < numTriangles2 && !(*intersectionFound); ++j) {
            if (shenIntersection(triangles1[i], triangles2[j])) {
                *intersectionFound = true;
            }
        }
    }
}

// Function to read triangles from an OFF file
vector<Triangle> readTrianglesFromOFF(const string& filename) {
    ifstream infile(filename);
    if (!infile) {
        cerr << "Error opening file: " << filename << endl;
        exit(1);
    }

    string header;
    infile >> header;
    if (header != "OFF") {
        cerr << "Not a valid OFF file." << endl;
        exit(1);
    }

    int numVertices, numFaces, numEdges;
    infile >> numVertices >> numFaces >> numEdges;

    vector<Vec3> vertices(numVertices);
    for (int i = 0; i < numVertices; ++i) {
        infile >> vertices[i][0] >> vertices[i][1] >> vertices[i][2];
    }

    vector<Triangle> triangles;
    for (int i = 0; i < numFaces; ++i) {
        int vertexCount, v1, v2, v3;
        infile >> vertexCount >> v1 >> v2 >> v3;
        if (vertexCount == 3) {
            triangles.push_back({vertices[v1], vertices[v2], vertices[v3]});
        }
    }

    return triangles;
}

int main() {
    auto start = chrono::high_resolution_clock::now();

    auto triangles1 = readTrianglesFromOFF("VH_M_ileocecal_valve.off");
    auto triangles2 = readTrianglesFromOFF("VH_M_transverse_colon.off");

    // Allocate device memory
    Triangle *d_triangles1, *d_triangles2;
    bool *d_intersectionFound;

    cudaMalloc(&d_triangles1, triangles1.size() * sizeof(Triangle));
    cudaMalloc(&d_triangles2, triangles2.size() * sizeof(Triangle));
    cudaMalloc(&d_intersectionFound, sizeof(bool));

    // Copy data to device
    cudaMemcpy(d_triangles1, triangles1.data(), triangles1.size() * sizeof(Triangle), cudaMemcpyHostToDevice);
    cudaMemcpy(d_triangles2, triangles2.data(), triangles2.size() * sizeof(Triangle), cudaMemcpyHostToDevice);

    bool intersectionFound = false;
    cudaMemcpy(d_intersectionFound, &intersectionFound, sizeof(bool), cudaMemcpyHostToDevice);

    // Launch kernel
    int blockSize = 256;
    int numBlocks = (triangles1.size() + blockSize - 1) / blockSize;
    triangleIntersectionKernel<<<numBlocks, blockSize>>>(d_triangles1, triangles1.size(),
                                                         d_triangles2, triangles2.size(),
                                                         d_intersectionFound);

    // Copy result back to host
    cudaMemcpy(&intersectionFound, d_intersectionFound, sizeof(bool), cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_triangles1);
    cudaFree(d_triangles2);
    cudaFree(d_intersectionFound);

    if (intersectionFound) {
        cout << "Triangles intersect!" << endl;
    } else {
        cout << "Triangles do not intersect." << endl;
    }

    auto end = chrono::high_resolution_clock::now();
    chrono::duration<double> elapsed = end - start;
    cout << "Time taken: " << elapsed.count() << " seconds" << endl;

    return 0;
}

Writing Shane.cu


In [15]:
!nvcc Shane.cu -o Shane --expt-relaxed-constexpr

Shane.cu(12): warning #177-D: variable "EPSILON" was declared but never referenced
  constexpr double EPSILON = 1e-12;
                   ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [16]:
!./Shane

Triangles do not intersect.
Time taken: 0.136621 seconds
